In [ ]:
!pip install tensorflow_addons

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Dropout, Conv1D, Flatten, GRU, MaxPooling1D, InputLayer, LeakyReLU, Embedding, Input
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [6]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks'

In [2]:
data = pd.read_parquet('full_dataset.parquet', engine='pyarrow')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [7]:
train

,label,session_id
0,2,6
1,1,13
2,2,21
3,2,22
4,4,32
...,...,...
208481,3,1606177
208482,2,1606199
208483,2,1606217
208484,1,1606226


In [8]:
test

,session_id
0,23
1,27
2,30
3,58
4,64
...,...
139346,1606165
139347,1606172
139348,1606196
139349,1606201


In [9]:
data.head()

,session_id,user_identifier,event_timestamp,event_category_idx_lv1,event_category_idx_lv2,event_category_idx_lv3,event_category_idx_lv4,tobi_timestamp
0,182576,0,2020-09-22 14:38:14,1_5,2,None,None,2020-09-24 02:03:53
1,182576,0,2020-09-22 14:38:14,1_4,2_9,3_5,None,2020-09-24 02:03:53
2,182576,0,2020-09-22 11:46:56,1_4,2_9,3_6,None,2020-09-24 02:03:53
3,182576,0,2020-09-22 11:23:50,1_4,2_9,3_12,None,2020-09-24 02:03:53
4,182576,0,2020-09-22 11:21:29,1_4,2_9,3_6,None,2020-09-24 02:03:53


In [10]:
data.sort_values(by=['session_id','event_timestamp'], inplace=True)

In [11]:
data['tobi_timestamp'] = (data['tobi_timestamp'] - min(data['tobi_timestamp'])).apply(lambda td: td.delta) * 1e-9

In [ ]:
minimum_date = data.groupby('user_identifier', sort=False).min(numeric_only=True)

In [ ]:
data

,session_id,user_identifier,event_timestamp,event_category_idx_lv1,event_category_idx_lv2,event_category_idx_lv3,event_category_idx_lv4,tobi_timestamp
4134595,6,237376,2020-07-15 08:47:31,1_0,2_3,None,None,4445182.0
4134594,6,237376,2020-07-15 20:08:52,1_0,2_2,None,None,4445182.0
4134593,6,237376,2020-07-15 20:09:20,1_0,2_4,None,None,4445182.0
4134592,6,237376,2020-07-15 20:09:31,1_0,2_4,None,None,4445182.0
4134591,6,237376,2020-07-15 20:10:15,1_0,2_3,None,None,4445182.0
...,...,...,...,...,...,...,...,...
5532705,1606217,701497,2020-06-18 11:25:00,1_5,1,None,None,1540260.0
5532706,1606217,701497,2020-06-18 11:25:00,1_3,2_6,None,None,1540260.0
5676991,1606224,871126,2020-06-01 00:00:00,1_2,2_1,3_2,None,123330.0
1431141,1606226,25526,2020-06-04 13:24:00,1_1,2_0,3_0,4_1,323692.0


In [ ]:
filt = data['event_category_idx_lv2'].apply(lambda s: s.isnumeric() if isinstance(s, str) else False)
to_drop = data[filt]

In [ ]:
data.drop(index=to_drop.index, inplace=True)

In [ ]:
grouped_by_idx = data.groupby(['event_category_idx_lv2', 'event_category_idx_lv3'])['event_category_idx_lv4'].unique().apply(lambda s: s[0] if s[0] == None else s).dropna()

In [ ]:
len_grouped_by_idx = grouped_by_idx.apply(lambda s: len(s))
drop_4th = []
for i in len_grouped_by_idx.index:
    if len_grouped_by_idx[i] == 1:
        drop_4th.append(i)
drop_4th = np.array(drop_4th)

In [ ]:
drop_3rd_and_4th = []
for i, j in drop_4th:
    if len(len_grouped_by_idx[i]) == 1:
        drop_3rd_and_4th.append(i)

In [ ]:
filt = data['event_category_idx_lv2'].isin(drop_3rd_and_4th)
data.loc[filt, ['event_category_idx_lv3', 'event_category_idx_lv4']] = None

In [ ]:
filt = (data['event_category_idx_lv2'].isin(drop_4th[:,0])) & (data['event_category_idx_lv3'].isin(drop_4th[:,1]))
data.loc[filt, 'event_category_idx_lv4'] = None

In [ ]:
baseline_date = pd.Timestamp(year=2020, month=4, day=14)

In [ ]:
data['event_timestamp'] = (data['event_timestamp'] - baseline_date).apply(lambda ts: ts.delta) * 1e-9 

In [ ]:
event_categories = ['event_category_idx_lv1', 'event_category_idx_lv2', 'event_category_idx_lv3', 'event_category_idx_lv4']
def filt(s):
    if isinstance(s, str):
        return s if s.isnumeric() else s[2:]
    return -1
for col in event_categories:
    data[col] = data[col].apply(filt)

In [ ]:
data

,session_id,user_identifier,event_timestamp,event_category_idx_lv1,event_category_idx_lv2,event_category_idx_lv3,event_category_idx_lv4,tobi_timestamp
4134595,6,237376,7980451.0,0,3,-1,-1,4445182.0
4134594,6,237376,8021332.0,0,2,-1,-1,4445182.0
4134593,6,237376,8021360.0,0,4,-1,-1,4445182.0
4134592,6,237376,8021371.0,0,4,-1,-1,4445182.0
4134591,6,237376,8021415.0,0,3,-1,-1,4445182.0
...,...,...,...,...,...,...,...,...
2883978,1606201,97349,14061100.0,4,9,5,-1,9890853.0
5532706,1606217,701497,5657100.0,3,6,-1,-1,1540260.0
5676991,1606224,871126,4147200.0,2,1,2,-1,123330.0
1431141,1606226,25526,4454640.0,1,0,0,1,323692.0


In [ ]:
training_data = data[data['session_id'].isin(train['session_id'])]

In [ ]:
testing_data = data[data['session_id'].isin(test['session_id'])]

In [ ]:
y_train, y_test_and_val = train_test_split(train, train_size=0.7, random_state=42)
y_test, y_val = train_test_split(y_test_and_val, train_size=0.5, random_state=42)

In [ ]:
for df in [y_train, y_test, y_val]:
    df.sort_values(by='session_id', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train_df = data[data['session_id'].isin(y_train['session_id'])]
test_df = data[data['session_id'].isin(y_test['session_id'])]
val_df = data[data['session_id'].isin(y_val['session_id'])]

In [ ]:
n_sequences = data['session_id'].nunique()
n_sequences

347837

In [ ]:
train_labels_by_user = get_user_data(train_df, y_train)

In [ ]:
train_labels_by_user

,0,1,2,3,4
0,0.0,16.0,0.0,7.0,11.0
1,1.0,6.0,0.0,7.0,11.0
2,2.0,10.0,0.0,4.0,2.0
3,3.0,9.0,0.0,4.0,9.0
4,4.0,0.0,12.0,0.0,1.0
...,...,...,...,...,...
124377,870984.0,0.0,1.0,0.0,0.0
124378,871090.0,0.0,1.0,0.0,0.0
124379,871093.0,0.0,0.0,0.0,1.0
124380,871095.0,0.0,0.0,0.0,1.0


In [ ]:
def reshape_inputs(dataset, users_history):
    n_sequences = dataset['session_id'].nunique()
    max_length = max(dataset['session_id'].value_counts())
    unique_users = users_history[0].unique()
    sequences = -np.ones((n_sequences, max_length, 7))
    user_data = np.zeros((n_sequences, 4))

    i = 0
    grouped = dataset.groupby('session_id').apply(np.array)
    for seq in grouped:
        l = len(seq)
        user = seq[0,1]
        sequences[i,:l] = seq[:,1:]
        if user in unique_users:
            user_data[i,:] = users_history.loc[users_history[0] == user, [1,2,3,4]]
        i += 1
    return sequences, user_data

In [ ]:
def reshape_sequences(dataset):
    n_sequences = dataset['session_id'].nunique()
    max_length = max(dataset['session_id'].value_counts())
    sequences = -np.ones((n_sequences, max_length, 7))

    i = 0
    grouped = dataset.groupby('session_id').apply(np.array)
    for seq in grouped:
        l = len(seq) 
        sequences[i,:l] = seq[:,1:] 
        i += 1
    return sequences

In [ ]:
def get_user_data(dataset, series):
    user_sessions = dataset.groupby('session_id')['user_identifier'].unique().apply(lambda s: s[0]).reset_index()
    user_sessions['label'] = series.sort_values(by='session_id').reset_index()['label']
    unique_users = np.sort(user_sessions['user_identifier'].unique())
    labels_by_user = user_sessions.groupby('user_identifier')['label'].value_counts()

    labels_count_by_user = np.zeros((124382,5), dtype=np.float32)
    j = 0
    for user in unique_users:
        labels_count_by_user[j,0] = user
        for i in labels_by_user[user].index:
            labels_count_by_user[j,i] = labels_by_user[user,i]
        j += 1
    
    return pd.DataFrame(labels_count_by_user)

In [ ]:
X_train_1, X_train_2 = reshape_inputs(train_df, train_labels_by_user)
X_test_1, X_test_2 = reshape_inputs(test_df, train_labels_by_user)
X_val_1, X_val_2 = reshape_inputs(val_df, train_labels_by_user)

In [ ]:
y_train = y_train['label'] - 1 # REMEMBER TO DO +1 WHEN SUBMITTING
y_test = y_test['label'] - 1
y_val = y_val['label'] - 1

In [ ]:
i = 0
for label in y_train:
    X_train_2[i,label] -= 1
    i += 1

In [ ]:
scaler = StandardScaler()
X_train_1 = scaler.fit_transform(X_train_1.reshape(-1, X_train_1.shape[-1])).reshape(X_train_1.shape)
X_test_1 = scaler.transform(X_test_1.reshape(-1, X_test_1.shape[-1])).reshape(X_test_1.shape)
X_val_1 = scaler.transform(X_val_1.reshape(-1, X_val_1.shape[-1])).reshape(X_val_1.shape)

In [ ]:
X_train_2 = scaler.fit_transform(X_train_2)
X_test_2 = scaler.transform(X_test_2)
X_val_2 = scaler.transform(X_val_2)

In [ ]:
X_train_2

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 1., 0., 1.]])

In [ ]:
X_train_2

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 1., 1., 0.],
       [2., 1., 0., 1.]])

In [ ]:
training = reshape_sequences(training_data)

In [ ]:
testing = reshape_sequences(testing_data)

In [ ]:
returning_users_test = pd.Series(X_test_1[:,0,0]).isin(X_train_1[:,0,0])
returning_users_val = pd.Series(X_val_1[:,0,0]).isin(X_train_1[:,0,0])   

In [ ]:
new_users_test, y_new_test = X_test[~returning_users_test], y_test[~returning_users_test]
new_users_val, y_new_val = X_val[~returning_users_val], y_val[~returning_users_val]

In [ ]:
new_users_train = X_train[:,:,1:]
new_users_test = new_users_test[:,:,1:]
new_users_val = new_users_val[:,:,1:]

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

In [ ]:
scaler = StandardScaler()
training = scaler.fit_transform(training.reshape(-1, training.shape[-1])).reshape(training.shape)
testing = scaler.transform(testing.reshape(-1, testing.shape[-1])).reshape(testing.shape)

In [ ]:
scaler.mean_

array([ 8.50004676e+04,  4.33559210e+06,  6.83401855e-01,  3.54139053e+00,
        2.76293911e-01, -7.16633548e-01,  3.23954465e+06])

In [ ]:
def build_model(filters=128, kernel_size=3):
    model = Sequential()
    n_layers = int(np.ceil(np.log2(30 / (kernel_size - 1))))
    dilation_schedule = [2**k for k in range(1, n_layers)]

    model.add(Conv1D(filters, kernel_size, padding='causal', dilation_rate=1, activation='relu', input_shape=(30, 7)))
    for rate in dilation_schedule:
        model.add(WeightNormalization(Conv1D(filters, kernel_size, padding='causal', dilation_rate=rate, activation='relu')))
        model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax'))

    opt = Adam(learning_rate=1e-3, amsgrad=True)
    model.compile(
        optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model

def build_model(filters=128, kernel_size=3):
    input2 = Input(shape=(4))

    input3 = Input(shape=(30,7))

    n_layers = int(np.ceil(np.log2(30 / (kernel_size - 1))))
    dilation_schedule = [2**k for k in range(1, n_layers)]

    conv = Conv1D(filters, kernel_size, padding='causal', dilation_rate=1, activation='relu', input_shape=(30,7))(input3)
    for rate in dilation_schedule:
        conv = WeightNormalization(Conv1D(filters, kernel_size, padding='causal', dilation_rate=rate, activation='relu'))(conv)
        conv = Dropout(0.3)(conv)
    flattened = Flatten()(conv)

    concatenated = concatenate([input2, flattened])
    dense = Dense(1024, activation='relu')(concatenated)
    dropout = Dropout(0.3)(dense)
    dense = Dense(2048, activation='relu')(dropout)
    dropout = Dropout(0.3)(dense)
    out = Dense(4, activation='softmax')(dropout)
    model = Model([input3, input2], out)

    opt = Adam(learning_rate=1e-3, amsgrad=True)
    model.compile(
        optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model

def build_model(filters=128, kernel_size=3): # BEST ONE
    input2 = Input(shape=(4))

    input3 = Input(shape=(30,7))

    n_layers = int(np.ceil(np.log2(30 / (kernel_size - 1))))
    dilation_schedule = [2**k for k in range(1, n_layers)]

    conv = Conv1D(filters, kernel_size, padding='causal', dilation_rate=1, activation='relu', input_shape=(30,7))(input3)
    for rate in dilation_schedule:
        conv = WeightNormalization(Conv1D(filters, kernel_size, padding='causal', dilation_rate=rate, activation='relu'))(conv)
        conv = Dropout(0.3)(conv)
    flattened = Flatten()(conv)

    concatenated = concatenate([input2, flattened])
    dense = Dense(1024, activation='relu')(concatenated)
    dropout = Dropout(0.3)(dense)
    dense = Dense(2048, activation='relu')(dropout)
    dropout = Dropout(0.3)(dense)
    out = Dense(4, activation='softmax')(dropout)
    model = Model([input3, input2], out)

    opt = Adam(learning_rate=1e-3, amsgrad=True)
    model.compile(
        optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model()

In [ ]:
model = KerasClassifier(build_model)
batch_sizes = [32, 64, 128, 256]
epochs = [20, 40, 60]
filters = [64, 128, 256, 512]
kernel_sizes = [2, 3, 4, 5, 6, 7, 8]
param_grid = {'kernel_size': kernel_sizes, 'epochs': epochs, 'batch_size': batch_sizes, 'filters': filters} # 'batch_size': batch_sizes 'filters': filters, 'kernel_size': kernel_sizes}

grid = GridSearchCV(model, param_grid=param_grid)
grid_result = grid.fit(new_users_train, y_train, cv=3)

In [ ]:
grid.best_params_

In [ ]:
def build_recurrent():
    model = Sequential()

    model.add(GRU(240, return_sequences=False, dropout=0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation='softmax'))

    opt = Adam(learning_rate=1e-3, amsgrad=True)
    model.compile(
        optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
history = model.fit([X_train_1, X_train_2], y_train, validation_data=([X_val_1, X_val_2], y_val), epochs=20, batch_size=64)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val[returning_users_val], y_val[returning_users_val]), epochs=20, batch_size=64)

In [ ]:
history = model.fit(training, labels, epochs=20, batch_size=64)

In [ ]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 30, 7)]      0                                            
__________________________________________________________________________________________________
conv1d_24 (Conv1D)              (None, 30, 128)      2816        input_14[0][0]                   
__________________________________________________________________________________________________
weight_normalization_18 (Weight (None, 30, 128)      98689       conv1d_24[0][0]                  
__________________________________________________________________________________________________
dropout_30 (Dropout)            (None, 30, 128)      0           weight_normalization_18[0][0]    
____________________________________________________________________________________________

In [ ]:
model.evaluate([X_test_1, X_test_2], y_test)

978/978 [==============================] - 4s 4ms/step - loss: 1.1170 - accuracy: 0.5188


[1.1170214414596558, 0.5188181400299072]

In [ ]:
model.evaluate([X_test_1[returning_users_test,:,:], X_test_2[returning_users_test,:]], y_test[(returning_users_test).to_numpy()])

254/254 [==============================] - 1s 4ms/step - loss: 0.9401 - accuracy: 0.6330


[0.9401163458824158, 0.6329551339149475]

In [ ]:
model.save_weights('weights_conv_best.h5')

In [ ]:
model.load_weights('weights_conv1.h5')

In [ ]:
prediction = np.argmax(model.predict(testing), axis=1) + 1

In [ ]:
prediction = prediction.astype(int)
np.savetxt('submission.txt', prediction)

In [ ]:
y_pred = np.argmax(model.predict([X_test_1,X_test_2]), axis=1)
confusion_matrix(y_test, y_pred)

In [ ]:
l = len(labels)
for i in range(4):
    print(f'samples from class {i}: {np.sum(labels == i) / l} %')

samples from class 0: 0.26656945790125 %
samples from class 1: 0.3656312654087085 %
samples from class 2: 0.1892597104841572 %
samples from class 3: 0.17853956620588432 %
